# 파이혜안 실습_과제 - 홍성희

## 파이혜안 불러오기

In [1]:
pip install pi-heaan

Note: you may need to restart the kernel to use updated packages.


In [2]:
import piheaan as heaan
from piheaan.math import approx

## 파라미터 설정

In [3]:
params = heaan.ParameterPreset.FGb
context = heaan.make_context(params) # context: 만들어진 동형암호 체계에 필요한 키
heaan.make_bootstrappable(context) # default는 bootstrap(재부팅)을 하지 않는 것. 근데 재부팅 가능하도록 설정.

## 키(비밀키, 공개키) 생성

In [4]:
key_dir_path = "./key_new"
secret_key = heaan.SecretKey(context) # 파이혜안은 대충 이렇게 돌아간다~를 알려주기 위한 시뮬레이터라서 키를 실제로 생성하진 않는다
secret_key.save("./secret_key.bin")
key_generator = heaan.KeyGenerator(context, secret_key)
key_generator.gen_common_keys() # 공개키(암호화), 곱셈, 컨쥬게이션, 로테이션, ,,,
key_generator.save(key_dir_path)
public_key = key_generator.keypack

In [5]:
secret_key

In [6]:
public_key

## 메시지 준비

In [7]:
log_slots = heaan.get_log_full_slots(context)
num_slots = 2**log_slots

In [8]:
# log_slots size 확인
log_slots

15

## Data 준비

In [9]:
import numpy as np
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

# original Data (X)
x_org = housing.data[:2**14]
# y True
yt = housing.target[:2**14]
# 분석에 사용할 X, MedInc
x1 = x_org[:, housing.feature_names.index('MedInc')]
# # y = w0*1 + w1*x 에서 (1, x)와 (w0, w1)의 내적으로 y를 구하는 형태를 활용하기 위해 (1,x)의 1 부분을 만든다
x0 = np.ones(2**14)

# w0, w1
w0 = np.ones(2**14)
w1 = np.ones(2**14)

# x_data = x_data.reshape(-1,1)
# x = np.insert(x_data, 0, 1.0, axis = 1)

## 메시지 준비

In [10]:
# log_slots 만큼의 크기를 가진 Message class를 만들고 
message_x0 = heaan.Message(log_slots)
message_x1 = heaan.Message(log_slots)
message_yt = heaan.Message(log_slots)
message_w0 = heaan.Message(log_slots)
message_w1 = heaan.Message(log_slots)

# 암호화하고자 하는 값을 message에 집어 넣음
for i in range(2**14):
    message_x0[i] = x0[i]
    message_x1[i] = x1[i]
    message_yt[i] = yt[i]
    message_w0[i] = w0[i]
    message_w1[i] = w1[i]

## Encrypt / Decrypt

In [11]:
# encryptor를 선언
encryptor = heaan.Encryptor(context)

# 빈 ciphertext를 만들고, encryptor를 이용해 message를 public_key로 encrypt한 걸 ciphertext에 저장
ciphertext_x0 = heaan.Ciphertext(context)
encryptor.encrypt(message_x0, public_key, ciphertext_x0)

ciphertext_x1 = heaan.Ciphertext(context)
encryptor.encrypt(message_x1, public_key, ciphertext_x1)

ciphertext_yt = heaan.Ciphertext(context)
encryptor.encrypt(message_yt, public_key, ciphertext_yt)

ciphertext_w0 = heaan.Ciphertext(context)
encryptor.encrypt(message_w0, public_key, ciphertext_w0)

ciphertext_w1 = heaan.Ciphertext(context)
encryptor.encrypt(message_w1, public_key, ciphertext_w1)

# decryptor를 선언
decryptor = heaan.Decryptor(context)

## Encode 

In [12]:
# endecoder 선언
endecoder = heaan.EnDecoder(context)

# encoding해서 plaintext 생성
ptxt_x0 = endecoder.encode(message_x0)
ptxt_x1 = endecoder.encode(message_x1)
ptxt_yt = endecoder.encode(message_yt)
ptxt_w0 = endecoder.encode(message_w0)
ptxt_w1 = endecoder.encode(message_w1)

## Linear Regression

In [13]:
# 변수 선언
iters = 2000
alpha = 0.01
M = len(x1)

# history 저장을 위해
history = np.zeros((0,2))

# 암호문 동형 연산 준비
evaluator = heaan.HomEvaluator(context, public_key)

In [14]:
def CheckBootstrap(ciphertextList:list):
    for c in ciphertextList:
        if c.level < 7:
            # print("before bootstrap {c}: ", c.level)
            evaluator.bootstrap(c, c, True)
            # print("bootstrap {c} level: ", c.level)

In [15]:
def GradientDescent(ciphertext_x:heaan.Ciphertext, ciphertext_yd:heaan.Ciphertext, ciphertext_w:heaan.Ciphertext, M:int, alpha:int):
    '''
    context, evaluator, log_slots는 미리 만들어져 있어야해요
    '''
    # x, yd의 element-wise 곱 xyd
    ciphertext_xyd = heaan.Ciphertext(context)
    evaluator.mult(ciphertext_x, ciphertext_yd, ciphertext_xyd)

    # xyd의 element 합
    for i in range(log_slots):
        ciphertext_temp = heaan.Ciphertext(context)
        evaluator.left_rotate(ciphertext_xyd, 2**i, ciphertext_temp)
        evaluator.add(ciphertext_xyd, ciphertext_temp, ciphertext_xyd)

    # g = xyd / M * alpha
    ciphertext_g = heaan.Ciphertext(context)
    evaluator.mult(ciphertext_xyd, 1/M, ciphertext_g)
    evaluator.mult(ciphertext_g, alpha, ciphertext_g)

    # w = w - g
    evaluator.sub(ciphertext_w, ciphertext_g, ciphertext_w)

    # bootstrap
    CheckBootstrap([ciphertext_x, ciphertext_yd, ciphertext_w, ciphertext_xyd, ciphertext_temp, ciphertext_g])
    
    return ciphertext_w

In [16]:
def GDHistory(ciphertext_yd:heaan.Ciphertext, history:np.ndarray, iter:int):
    '''
    decryptor, evaluator, secret_key가 있어야 작동해요
    '''
    # yd**2
    ciphertext_loss = heaan.Ciphertext(context)
    evaluator.mult(ciphertext_yd, ciphertext_yd, ciphertext_loss)

    # yd**2의 summation
    for i in range(log_slots):
        ciphertext_temp = heaan.Ciphertext(context)
        evaluator.left_rotate(ciphertext_loss, 2**i, ciphertext_temp)
        evaluator.add(ciphertext_loss, ciphertext_temp, ciphertext_loss)
    
    ### bootstrap
    CheckBootstrap([ciphertext_yd, ciphertext_loss, ciphertext_temp])
    
    # yd**2 summation의 1/2M
    evaluator.mult(ciphertext_loss, 1/num_slots, ciphertext_loss)
    evaluator.mult(ciphertext_loss, 1/2, ciphertext_loss)

    # loss decrypt
    message_loss_out = heaan.Message(log_slots)
    decryptor.decrypt(ciphertext_loss, secret_key, message_loss_out)

    # loss 저장
    history = np.vstack((history, np.array([iter, message_loss_out[0]])))

    print("iter:", iter, "loss:", message_loss_out[0])

    return history


In [17]:
for k in range(iters):
    
    # w1x1 = w1*x1
    ciphertext_w1x1 = heaan.Ciphertext(context)
    evaluator.mult(ciphertext_w1, ciphertext_x1, ciphertext_w1x1)

    # yp = w0 + w1x
    ciphertext_yp = heaan.Ciphertext(context)
    evaluator.add(ciphertext_w0, ciphertext_w1x1, ciphertext_yp)
    
    # yd = yp-yt
    ciphertext_yd = heaan.Ciphertext(context)
    evaluator.sub(ciphertext_yp, ciphertext_yt, ciphertext_yd)

    # bootstrap
    CheckBootstrap([ciphertext_w1x1, ciphertext_w1, ciphertext_x1, ciphertext_yp, ciphertext_w0, ciphertext_yt, ciphertext_yd])

    # Gradient Descent
    ciphertext_w0 = GradientDescent(ciphertext_x0, ciphertext_yd, ciphertext_w0, M, alpha)
    ciphertext_w1 = GradientDescent(ciphertext_x1, ciphertext_yd, ciphertext_w1, M, alpha)

    # Decrypt하여 history 저장 및 출력
    if k%100 == 0:
        history = GDHistory(ciphertext_yd, history, k)

iter: 0 loss: (2.415049488170238+0j)
iter: 100 loss: (0.19301013858348706+0j)
iter: 200 loss: (0.19788162525859912+0j)
iter: 300 loss: (0.20325438671712995+0j)
iter: 400 loss: (0.20862625793303527+0j)
iter: 500 loss: (0.21371097957974303+0j)
iter: 600 loss: (0.21836066461096265+0j)
iter: 700 loss: (0.22251420945248834+0j)
iter: 800 loss: (0.22616325015781305+0j)
iter: 900 loss: (0.22932993830805762+0j)
iter: 1000 loss: (0.23205264175233697+0j)
iter: 1100 loss: (0.23437692666109133+0j)
iter: 1200 loss: (0.23635003188887826+0j)
iter: 1300 loss: (0.23801762930647308+0j)
iter: 1400 loss: (0.2394220603347293+0j)
iter: 1500 loss: (0.24060150822553564+0j)
iter: 1600 loss: (0.2415897480211219+0j)
iter: 1700 loss: (0.24241623921584501+0j)
iter: 1800 loss: (0.24310640886362542+0j)
iter: 1900 loss: (0.24368202816044224+0j)


## Graph

In [29]:
history_f_iter = []
history_f_loss = []

for i in range(len(history.T[0])):
    history_f_iter.append(history.T[0][i].real)
    history_f_loss.append(history.T[1][i].real)
    
print(history_f_iter)
print(history_f_loss)


[0.0, 100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700.0, 800.0, 900.0, 1000.0, 1100.0, 1200.0, 1300.0, 1400.0, 1500.0, 1600.0, 1700.0, 1800.0, 1900.0]
[2.415049488170238, 0.19301013858348706, 0.19788162525859912, 0.20325438671712995, 0.20862625793303527, 0.21371097957974303, 0.21836066461096265, 0.22251420945248834, 0.22616325015781305, 0.22932993830805762, 0.23205264175233697, 0.23437692666109133, 0.23635003188887826, 0.23801762930647308, 0.2394220603347293, 0.24060150822553564, 0.2415897480211219, 0.24241623921584501, 0.24310640886362542, 0.24368202816044224]


In [30]:
import plotly.graph_objects as go
#그래프 생성
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=history_f_iter,
    y=history_f_loss
))

fig.show()

### 연습

In [18]:
# 내적을 어떻게 계산하지? 궁금해서 해보는 테스트

# 데이터 준비 
test_1 = [i for i in range(2**14)] # [0,1,2,3,...]
test_2 = [2 for i in range(2**14)] # [2,2,2,2,...]

# 메시지 준비
message_t1 = heaan.Message(log_slots)
message_t2 = heaan.Message(log_slots)

for i in range(2**14):
    message_t1[i] = test_1[i]
    message_t2[i] = test_2[i]

# encryption
ciphertext_t1 = heaan.Ciphertext(context)
encryptor.encrypt(message_t1, public_key, ciphertext_t1)
ciphertext_t2 = heaan.Ciphertext(context)
encryptor.encrypt(message_t2, public_key, ciphertext_t2)

# encoding
ptxt_t1 = endecoder.encode(message_t1)
ptxt_t2 = endecoder.encode(message_t2)

# 결과 변수 선언
ciphertext_t3 = heaan.Ciphertext(context)

# evaluate
evaluator.mult(ciphertext_t1, ciphertext_t2, ciphertext_t3)

# decrypt
decryptor = heaan.Decryptor(context)
message_out_t3 = heaan.Message(log_slots)
decryptor.decrypt(ciphertext_t3, secret_key, message_out_t3)

print(message_out_t3)

[ (0.000000+0.000000j), (2.000000+0.000000j), (4.000000+0.000000j), (6.000000+0.000000j), (8.000000+0.000000j), ..., (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j) ]


In [ ]:
w = np.ones(2)
wtest = w-1
wtest

array([0., 0.])

In [ ]:
xttest = np.array([[1,1],[1,2],[1,3]])
xttest.T
yttest = np.array([5,10,15])
xttest.T@yttest

array([30, 70])

In [22]:
for i in range(20000):
    evaluator.mult(ciphertext_t1, ciphertext_t2, ciphertext_t3)
    
    if ciphertext_t1.level < 7:
        print("before bootstrap t1: ", ciphertext_t1.level)
        evaluator.bootstrap(ciphertext_t1, ciphertext_t1, True)
        print("bootstrap t1 level: ", ciphertext_t1.level)
    if ciphertext_t2.level < 7:
        print("before bootstrap t2: ", ciphertext_t2.level)
        evaluator.bootstrap(ciphertext_t2, ciphertext_t2, True)
        print("bootstrap t2 level: ", ciphertext_t2.level)
    if ciphertext_t3.level < 7:
        print("before bootstrap t3: ", ciphertext_t3.level)
        evaluator.bootstrap(ciphertext_t3, ciphertext_t3, True)
        print("bootstrap t3 level: ", ciphertext_t3.level)    